In [30]:
import random
import csv
from collections import defaultdict
import pandas as pd

In [31]:
# PARÁMETROS
# los deduce de los slots

# depende si quiero usar los slots de bloques de hora y media o los de 45 minutos
#df_slots = pd.read_csv("datos/slots.csv")
df_slots = pd.read_csv("../datos/slots_divididos_dic_2024.csv")

DIAS = df_slots['dia'].unique().tolist()
SLOTS = df_slots['slot'].unique().tolist()

print(DIAS)
print(SLOTS)

W1, W2, W3 = 10, 3, 0  # pesos del costo (días, disponibilidad 2, huecos)

['Lunes', 'Martes', 'Miércoles', 'Jueves']
[1, 2, 3, 4, 5, 6, 7, 8]


In [32]:
# lo cambio a mano
# DIAS = ['Lunes', 'Viernes']
# SLOTS=[1, 2, 3, 4, 5, 6, 7, 8]

## Carga los datos

In [33]:
def cargar_asignaciones(path):
    asign = {}
    with open(path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            asign[row['alumno']] = [row['jurado1'], row['jurado2'], row['jurado3']]
    return asign

def cargar_disponibilidad(path):
    disp = defaultdict(lambda: defaultdict(dict))
    with open(path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            jur = row['jurado']
            dia = row['dia']
            slot = int(row['slot'])

            #ignoro el "si, si fuera necesario"
            if (row['disponibilidad']==1):
                row['disponibilidad']=2
                
            disp[jur][dia][slot] = int(row['disponibilidad'])
    return disp

In [34]:
asignaciones = cargar_asignaciones("../datos/asignaciones_dic_2024.csv")
disponibilidad = cargar_disponibilidad(f"../datos/disponibilidad_dic_2024.csv")

## Tabu search

In [35]:
# --- FUNCIÓN DE COSTO ---

def calcular_costo_total(solucion, asignaciones, disponibilidad):
    dias_usados = len(set(d for d, s in solucion.values()))

    uso_dispon2 = 0
    huecos = 0

    # Penalizar uso de disponibilidad tipo 2 y jurados no disponibles
    for alumno, (dia, slot) in solucion.items():
        for jur in asignaciones[alumno]:
            val = disponibilidad[jur][dia].get(slot, 0)
            if val == 0:
                return 1e9  # prohibido
            if val == 2:
                uso_dispon2 += 1

    # Penalizar huecos dentro de cada día
    for dia in DIAS:
        slots_ocupados = sorted([s for d, s in solucion.values() if d == dia])
        if len(slots_ocupados) > 1:
            for a, b in zip(slots_ocupados[:-1], slots_ocupados[1:]):
                if b - a > 1:
                    huecos += b - a - 1

    return W1 * dias_usados + W2 * uso_dispon2 + W3 * huecos

# --- GENERACIÓN DE VECINOS ---

def generar_vecindarios(solucion):
    vecindarios = []
    alumnos = list(solucion.keys())
    for alumno in alumnos:
        dia_actual, slot_actual = solucion[alumno]
        for dia in DIAS:
            for slot in SLOTS:
                if (dia, slot) not in solucion.values():
                    nueva = solucion.copy()
                    nueva[alumno] = (dia, slot)
                    movimiento = (alumno, dia_actual, slot_actual, dia, slot)
                    vecindarios.append((nueva, movimiento))
    return vecindarios

# --- BÚSQUEDA TABÚ ---

def busqueda_tabu(asignaciones, disponibilidad, num_iteraciones=500, tamanio_lista_tabu=20):
    # solución inicial heurística: primer horario disponible para cada alumno
    solucion_actual = {}
    usados = set()
    for alumno in asignaciones.keys():
        asignado = False
        for d in DIAS:
            for s in SLOTS:
                if (d, s) not in usados and all(disponibilidad[j][d].get(s, 0) != 0 for j in asignaciones[alumno]):
                    solucion_actual[alumno] = (d, s)
                    usados.add((d, s))
                    asignado = True
                    break
            if asignado:
                break

    mejor_solucion = solucion_actual.copy()
    mejor_costo = calcular_costo_total(mejor_solucion, asignaciones, disponibilidad)

    lista_tabu = []

    print("Solución inicial:", mejor_solucion)
    print("Costo inicial:", mejor_costo)

    for iteracion in range(num_iteraciones):
        vecindarios = generar_vecindarios(solucion_actual)

        mejor_vecindario = None
        mejor_movimiento = None
        mejor_costo_vecino = float('inf')

        for vecino, movimiento in vecindarios:
            costo_vecino = calcular_costo_total(vecino, asignaciones, disponibilidad)

            # criterio de aspiración
            if (movimiento not in lista_tabu) or (costo_vecino < mejor_costo):
                if costo_vecino < mejor_costo_vecino:
                    mejor_vecindario = vecino
                    mejor_movimiento = movimiento
                    mejor_costo_vecino = costo_vecino

        if mejor_vecindario is None:
            break  # no hay vecinos válidos

        solucion_actual = mejor_vecindario.copy()

        # actualizar lista tabú
        lista_tabu.append(mejor_movimiento)
        if len(lista_tabu) > tamanio_lista_tabu:
            lista_tabu.pop(0)

        # actualizar mejor solución global
        if mejor_costo_vecino < mejor_costo:
            mejor_solucion = mejor_vecindario.copy()
            mejor_costo = mejor_costo_vecino

        if iteracion % 20 == 0:
            print(f"Iteración {iteracion}: costo actual = {mejor_costo}")

    return mejor_solucion, mejor_costo


In [36]:
mejor_sol, mejor_costo = busqueda_tabu(asignaciones, disponibilidad)

print("\n=== MEJOR ASIGNACIÓN ENCONTRADA ===")
for alumno, (dia, slot) in mejor_sol.items():
    print(f"{alumno:10s} → {dia} (slot {slot})  Costo total: {mejor_costo}")

Solución inicial: {'Fernandez': ('Lunes', 1), 'Villarraza': ('Lunes', 2), 'Alianak': ('Lunes', 3), 'Valdez': ('Lunes', 4), 'Acerbo': ('Lunes', 5), 'Baffo': ('Lunes', 6), 'Bampini Basualdo': ('Lunes', 7), 'Vásquez González': ('Martes', 1), 'Saraco': ('Martes', 2), 'Paz': ('Martes', 3), 'Monroy Guerrero': ('Martes', 5), 'Mendoza': ('Martes', 6), 'Anselmi Hernandez': ('Miércoles', 3), 'Ferrán': ('Miércoles', 4), 'Brumovsky': ('Miércoles', 5), 'Andrioli Villa': ('Miércoles', 6), 'Carvallo Murga': ('Miércoles', 7), 'Crivos Gandini': ('Jueves', 3), 'Bazet': ('Jueves', 7)}
Costo inicial: 160
Iteración 0: costo actual = 154
Iteración 20: costo actual = 151
Iteración 40: costo actual = 151
Iteración 60: costo actual = 151
Iteración 80: costo actual = 151
Iteración 100: costo actual = 151
Iteración 120: costo actual = 151
Iteración 140: costo actual = 151
Iteración 160: costo actual = 151
Iteración 180: costo actual = 151
Iteración 200: costo actual = 151
Iteración 220: costo actual = 151
Iterac

In [37]:
# mejor_sol: diccionario alumno -> (dia, slot)
# mejor_costo: valor total de la penalización

filas = []
for alumno, (dia, slot) in mejor_sol.items():
    filas.append({'Alumno': alumno, 'Día': dia, 'Slot': slot})

#df['Costo total'] = mejor_costo

df = pd.DataFrame(filas)

# Ordenamos primero por día, luego por slot
dias_orden = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']
df['Día'] = pd.Categorical(df['Día'], categories=dias_orden, ordered=True)
df = df.sort_values(['Día', 'Slot'])

In [38]:
# recupera toda la info para mostrar el resultado final
df_merged = df.merge(
    df_slots[['dia', 'slot', 'rango_horario']],
    left_on=['Día', 'Slot'],
    right_on=['dia', 'slot'],
    how='left'
)
df_merged = df_merged.drop(columns=['dia', 'slot'])
df_merged['jurados'] = df_merged['Alumno'].map(lambda x: ', '.join(asignaciones.get(x, [])))
df_final = df_merged[['Día', 'rango_horario', 'Alumno', 'jurados', 'Slot']]
df_final = df_final.sort_values(by=['Día', 'Slot']).reset_index(drop=True)
df_final = df_final[['Día', 'rango_horario', 'Alumno', 'jurados']]

In [39]:
print("\n=== MEJOR ASIGNACIÓN ENCONTRADA ===")
print(df_final.to_string(index=False))


=== MEJOR ASIGNACIÓN ENCONTRADA ===
      Día rango_horario            Alumno                                                                        jurados
   Jueves 17:30 a 18:15    Crivos Gandini                          pbos@fi.uba.ar, evolentini@gmail.com, jcruz@fi.uba.ar
   Jueves 20:30 a 21:15             Bazet                      alicialmon@gmail.com, vaubin@unlam.edu.ar, cad@uns.edu.ar
    Lunes 16:45 a 17:30        Villarraza    lmarianocampos@gmail.com, emilianorm@gmail.com, rraulemilioromero@gmail.com
    Lunes 17:30 a 18:15           Alianak farfan.roberto.f@gmail.com, gerardox2000@gmail.com, denisjorgegenero@gmail.com
    Lunes 18:15 a 19:00            Valdez      facundolucianna@gmail.com, maxit1992@gmail.com, rodolfopaganini@gmail.com
    Lunes 19:00 a 19:45            Acerbo           bureuclara@gmail.com, maxit1992@gmail.com, rodolfopaganini@gmail.com
    Lunes 20:30 a 21:15  Bampini Basualdo         roberto.axt@gmail.com, javifanelli@gmail.com, gabrielcarutti@gmail